### Tomatoes Classification 
We try to classify tomatoes in one of these category - damaged, old, ripe, unripe

We make use of this kaggle dataset - https://www.kaggle.com/datasets/enalis/tomatoes-dataset/data

The dataset is available in this folder structure:
```
/kaggle/input/tomatoes-dataset/content/ieee-mbl-cls/{train or val}/{class-of-tomato}/{some-name}.jpg
```

<ins>**_Train model_**</ins>

To train a model, we'll need `DataLoaders`, which is an object that contains a training set (the images used to create a model) and a validation set (the images used to check the accuracy of a model -- not used during training). In `fastai` we can create that easily using a `DataBlock`, and view sample images from it:



In [1]:
# import libraries
from fastcore.all import *
from fastai.vision.all import *

# get the path
path = Path('/kaggle/input/tomatoes-dataset/content/ieee-mbl-cls')

# create dataloader
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=GrandparentSplitter(train_name='train', valid_name='val'), # since the folder structure is "{train or val}/{class-of-tomato}/{some-name}.jpg"
    get_y=parent_label, # since parent folder name is the category of tomatoes - {class-of-tomato}/{tomato-file}.jpg
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)

ModuleNotFoundError: No module named 'fastcore'

Now we're ready to train our model. The fastest widely used computer vision model is `resnet18`. You can train this in a few minutes, even on a CPU! (On a GPU, it generally takes under 10 seconds...)

`fastai` comes with a helpful `fine_tune()` method which automatically uses best practices for fine tuning a pre-trained model, so we'll use that.

In [ ]:
# pass dataloader to learner
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

Generally when I run this I see 100% accuracy on the validation set (although it might vary a bit from run to run).

`Fine-tuning` a model means that we're starting with a model someone else has trained using some other dataset (called the pretrained model), and adjusting the weights a little bit so that the model learns to recognise your particular dataset. In this case, the pretrained model was trained to recognise photos in imagenet, and widely-used computer vision dataset with images covering 1000 categories).

Check how the model performs of validation set.

In [ ]:
# check result of learner
learn.show_results()

Lets test the model by uploading custom image of tomato.

In [ ]:
from ipywidgets import *
uploader = widgets.FileUpload()
uploader

In [ ]:
# see which image you have uploaded
img = PILImage.create(uploader.data[0])
img.to_thumb(128,128)

Now you can pass the uploaded file to the model. Make sure that it is a clear photo of a tomato, and not a line drawing, cartoon, or similar. The notebook will tell you which kind of tomato is it, and how confident it is. Hopefully, you'll find that your model did a great job:

In [ ]:
is_tomato,_,probs = learn.predict(img)
print(f"Which kind of tomato is it?: {is_tomato}.")
print(f"Confidence level of this Prediction: {probs[0]:.4f}")

Try with various example images see how it performs.